In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# For the dashboard


# Overview

In this notebook I have created a dashboard giving an overview of the problems in Zürich. The map shows the location of all problems, what the problem is and the status of the problem. The user can filter by the type of problem, to see only the problems of interest, e.g. only the problems with the type "Graffiti" that are still open.

### Why?/Usecases

Citizens can use this dashboard to see what problems there are in Zurich, to stay up to date.

If a citizen is planning a late picnic in Irchel Park, they can check the dashboard to see if there are any problems in the park, e.g.:

if the lighting is broken, they might have to bright a lamp themselves.
there was one (albeit erroneous) report of a type of mosquito that can carry the dangerous Dengue fever, so they might want to choose a different park.


Benevolent citizens can filter by cases of graffiti that are not fixed yet and clean it up themselves.

If you are moving to Wiedikon and don't know much about the area, you can check the dashboard to see what problems there are in the area, and if they are being fixed.


For the people behind "Züri wie neu", this dashboard could maybe offer a different overview of the city's problems.


### Data

This data set is from "Züri wie neu" (Zurich like new), a program where citizens can report problems or flaws in the city. Stadt Zurich will then respond to the report and ideally fix the problem.

The (important) columns in the data set are:

coordinates (e,n): the coordinates of the problem in Swiss (east, north) format

service_code: the type of problem in 10 categories

status: the status of the problem

requested_datetime: the date and time the problem was reported

updated_datetime: the date and time the problem was updated. If updated to "fixed" it indicates the time it took to fix the problem.

detail: a description of the problem from the citizen

media_url: a link to a picture of the problem uploaded by the citizen


### Analysis

The goal is to find some interesting insights in the data set and put it into a dashboard. The dashboard will be created using Plotly Dash.

### Dashboard Goals

The dashboard should:

Have a map of Zurich with the problems marked on it. The user should be able to select a problem and see the type, status, and when it was reported and updated.

Have a dropdown menu to filter by the type of problem.



## Inspect data

In [2]:
data_raw = pd.read_csv('C:/Users/simon/Desktop/Code/Zuri_wie_neu_dashboard/ZWN_data/data/stzh_zwn_meldungen_p.csv')

data_raw.head()

,objectid,service_request_id,e,n,service_code,status,detail,media_url,interface_used,requested_datetime,agency_sent_datetime,updated_datetime,service_notice,title,description,service_name,url,geometry
0,1,1,2678968.0,1247548.0,Strasse/Trottoir/Platz,fixed - council,Auf dem Asphalt des Bürgersteigs hat es eine E...,NaN,Web interface,20130314151615,2.013040e+13,20130412075930,Diese Reparatur wird von uns in den kommenden ...,Auf dem Asp,Auf dem Asp: Auf dem Asphalt des Bürgersteigs ...,Strasse/Trottoir/Platz,https://www.zueriwieneu.ch/report/1,POINT (2678968 1247548)
1,2,2,2680746.0,1249916.0,Strasse/Trottoir/Platz,fixed - council,Vermessungspunkt ist nicht mehr bündig mit dem...,NaN,Web interface,20130314151757,2.013033e+13,20130412080022,Diese Reparatur wird von uns in den kommenden ...,Vermessungs,Vermessungs: Vermessungspunkt ist nicht mehr b...,Strasse/Trottoir/Platz,https://www.zueriwieneu.ch/report/2,POINT (2680746 1249916)
2,3,4,2684605.0,1251431.0,Strasse/Trottoir/Platz,fixed - council,Beim Trottoir sind einige Randsteine defekt un...,https://www.zueriwieneu.ch/photo/4.0.jpeg?bfbb...,Web interface,20130315091416,2.013032e+13,20130412080810,Diese Reparatur wird von uns in den kommenden ...,Beim Trotto,Beim Trotto: Beim Trottoir sind einige Randste...,Strasse/Trottoir/Platz,https://www.zueriwieneu.ch/report/4,POINT (2684605 1251431)
3,4,5,2681754.0,1250376.0,Strasse/Trottoir/Platz,fixed - council,Auf dem Parkplatz beim Waidspital sind einige ...,https://www.zueriwieneu.ch/photo/5.0.jpeg?e309...,Web interface,20130315091715,2.013032e+13,20130412080905,Diese Reparatur wird von uns in den kommenden ...,Auf dem Par,Auf dem Par: Auf dem Parkplatz beim Waidspital...,Strasse/Trottoir/Platz,https://www.zueriwieneu.ch/report/5,POINT (2681754 1250376)
4,5,6,2683094.0,1247762.0,Abfall/Sammelstelle,fixed - council,Arbeitskiste ist rund herum verschmiert,https://www.zueriwieneu.ch/photo/6.0.jpeg?8e65...,Web interface,20130315103653,2.013042e+13,20130423135033,Dieses Graffiti wird von uns in den kommenden ...,Arbeitskist,Arbeitskist: Arbeitskiste ist rund herum versc...,Abfall/Sammelstelle,https://www.zueriwieneu.ch/report/6,POINT (2683094 1247762)


## Data preprocessing

In [3]:
# Remove duplicate column
(data_raw['service_name'] == data_raw['service_code']).all()
data = data_raw.drop('service_code', axis=1)

#Fill NaNs in messages
data['service_notice'].fillna('No response to report.', inplace=True)
data['detail'].fillna('No message in report.', inplace=True)

In [4]:
# Convert timestamp to datetime format
date_columns = ['requested_datetime', 'agency_sent_datetime', 'updated_datetime']

for col in date_columns:
    data[col] = pd.to_datetime(data[col], format='%Y%m%d%H%M%S')

data[date_columns]

,requested_datetime,agency_sent_datetime,updated_datetime
0,2013-03-14 15:16:15,2013-04-04 07:25:05,2013-04-12 07:59:30
1,2013-03-14 15:17:57,2013-03-26 14:05:05,2013-04-12 08:00:22
2,2013-03-15 09:14:16,2013-03-15 09:55:05,2013-04-12 08:08:10
3,2013-03-15 09:17:15,2013-03-20 10:05:05,2013-04-12 08:09:05
4,2013-03-15 10:36:53,2013-04-22 18:25:05,2013-04-23 13:50:33
...,...,...,...
54140,2024-09-04 16:29:01,2024-09-04 16:32:05,2024-09-04 19:00:15
54141,2024-09-04 16:43:28,2024-09-04 16:47:04,2024-09-04 19:14:00
54142,2024-09-04 16:46:14,2024-09-04 17:22:04,2024-09-04 17:22:04
54143,2024-09-04 16:51:49,2024-09-04 16:52:05,2024-09-04 16:53:07


In [5]:
data['media_url'].fillna('No media', inplace=True)

In [6]:
# Add time it took to fix the problem
data[data['status'] == 'fixed - council'][['requested_datetime', 'updated_datetime']]
data.loc[:,'service_time'] = np.zeros(data.shape[0])
data.loc[:,'service_time'] = data['updated_datetime'] - data['requested_datetime']
data.loc[:,'service_time'] = data['service_time'].dt.total_seconds() / 60 / 60
# Add report_day as a column
data.loc[:,'report_month'] = data.loc[:,'requested_datetime'].dt.month
# Add report_year
data.loc[:,'report_year'] = data.loc[:,'requested_datetime'].dt.year

# Ensure years and weekdays are ordered for plotting later
years = sorted(data['requested_datetime'].dt.year.unique())
months = sorted(data['requested_datetime'].dt.month.unique())#, key=lambda x: ['January', 'February', 'March', 'April', 'May', 'June', 'July', 'August', 'September', 'October', 'November', 'December'].index(x))
#weekdays = sorted(data['requested_datetime'].dt.day_name().unique(), key=lambda x: ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday'].index(x))


Converting the coordinates from the swiss east and north format to latitude and longitude.

In [7]:
from pyproj import Transformer

# Define the transformer for converting CH1903+ / LV95 to WGS84
transformer = Transformer.from_crs("EPSG:2056", "EPSG:4326", always_xy=True)


# Function to transform coordinates
def transform_coordinates(row):
    longitude, latitude = transformer.transform(row['e'], row['n'])
    return pd.Series([longitude, latitude], index=['longitude', 'latitude'])

# Apply the function to each row in the DataFrame
data[['longitude', 'latitude']] = data.apply(transform_coordinates, axis=1)


In [8]:
'1d1d<br/>'.replace('<br/>','\n')

'1d1d\n'

In [9]:
# Clean up SZ answers to reports
# clean text in service_note

data.loc[:,'service_notice'] = [msg.replace('<br/>', '\n ') for msg in data.loc[:,'service_notice'].values]
data.loc[:,'detail']         = [msg.replace('<br/>', '\n ') for msg in data.loc[:,'detail'].values]

data

,objectid,service_request_id,e,n,status,detail,media_url,interface_used,requested_datetime,agency_sent_datetime,...,title,description,service_name,url,geometry,service_time,report_month,report_year,longitude,latitude
0,1,1,2678968.0,1247548.0,fixed - council,Auf dem Asphalt des Bürgersteigs hat es eine E...,No media,Web interface,2013-03-14 15:16:15,2013-04-04 07:25:05,...,Auf dem Asp,Auf dem Asp: Auf dem Asphalt des Bürgersteigs ...,Strasse/Trottoir/Platz,https://www.zueriwieneu.ch/report/1,POINT (2678968 1247548),688.720833,3,2013,8.484226,47.374038
1,2,2,2680746.0,1249916.0,fixed - council,Vermessungspunkt ist nicht mehr bündig mit dem...,No media,Web interface,2013-03-14 15:17:57,2013-03-26 14:05:05,...,Vermessungs,Vermessungs: Vermessungspunkt ist nicht mehr b...,Strasse/Trottoir/Platz,https://www.zueriwieneu.ch/report/2,POINT (2680746 1249916),688.706944,3,2013,8.508193,47.395119
2,3,4,2684605.0,1251431.0,fixed - council,Beim Trottoir sind einige Randsteine defekt un...,https://www.zueriwieneu.ch/photo/4.0.jpeg?bfbb...,Web interface,2013-03-15 09:14:16,2013-03-15 09:55:05,...,Beim Trotto,Beim Trotto: Beim Trottoir sind einige Randste...,Strasse/Trottoir/Platz,https://www.zueriwieneu.ch/report/4,POINT (2684605 1251431),670.898333,3,2013,8.559589,47.408259
3,4,5,2681754.0,1250376.0,fixed - council,Auf dem Parkplatz beim Waidspital sind einige ...,https://www.zueriwieneu.ch/photo/5.0.jpeg?e309...,Web interface,2013-03-15 09:17:15,2013-03-20 10:05:05,...,Auf dem Par,Auf dem Par: Auf dem Parkplatz beim Waidspital...,Strasse/Trottoir/Platz,https://www.zueriwieneu.ch/report/5,POINT (2681754 1250376),670.863889,3,2013,8.521627,47.399131
4,5,6,2683094.0,1247762.0,fixed - council,Arbeitskiste ist rund herum verschmiert,https://www.zueriwieneu.ch/photo/6.0.jpeg?8e65...,Web interface,2013-03-15 10:36:53,2013-04-22 18:25:05,...,Arbeitskist,Arbeitskist: Arbeitskiste ist rund herum versc...,Abfall/Sammelstelle,https://www.zueriwieneu.ch/report/6,POINT (2683094 1247762),939.227778,3,2013,8.538891,47.375455
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
54140,54141,60666,2681646.0,1250853.0,fixed - council,Bei Waldspaziergang durch Tigermückenschwarm v...,https://www.zueriwieneu.ch/photo/60666.0.jpeg?...,iOS,2024-09-04 16:29:01,2024-09-04 16:32:05,...,Bei Waldspa,Bei Waldspa: Bei Waldspaziergang durch Tigermü...,Schädlinge,https://www.zueriwieneu.ch/report/60666,POINT (2681646 1250853),2.520556,9,2024,8.520284,47.403434
54141,54142,60668,2681290.0,1248242.0,fixed - council,Sehr geehrte Damen und Herren \n \n Sie wer...,https://www.zueriwieneu.ch/photo/60668.0.jpeg?...,iOS,2024-09-04 16:43:28,2024-09-04 16:47:04,...,Sehr geehrt,Sehr geehrt: Sehr geehrte Damen und Herren <br...,Schädlinge,https://www.zueriwieneu.ch/report/60668,POINT (2681290 1248242),2.508889,9,2024,8.515094,47.379997
54142,54143,60669,2680724.0,1245156.0,confirmed,"Flacher Wassertrog, ursprünglich zur Schuhrein...",No media,Web interface,2024-09-04 16:46:14,2024-09-04 17:22:04,...,Flacher Was,"Flacher Was: Flacher Wassertrog, ursprünglich ...",Grünflächen/Spielplätze,https://www.zueriwieneu.ch/report/60669,POINT (2680724 1245156),0.597222,9,2024,8.507043,47.352313
54143,54144,60670,2682008.0,1247425.0,fixed - council,Diese KACK Velo versperrt seit Wochen den Velo...,https://www.zueriwieneu.ch/photo/60670.0.jpeg?...,iOS,2024-09-04 16:51:49,2024-09-04 16:52:05,...,Diese KACK,Diese KACK : Diese KACK Velo versperrt seit Wo...,Abfall/Sammelstelle,https://www.zueriwieneu.ch/report/60670,POINT (2682008 1247425),0.021667,9,2024,8.524451,47.372560


In [10]:
import pandas as pd
from shapely.wkt import loads
from shapely.geometry import Point
from pyproj import Transformer

# Define the transformer for converting CH1903+ / LV95 to WGS84
transformer = Transformer.from_crs("EPSG:2056", "EPSG:4326", always_xy=True)

# Function to transform coordinates
def transform_coordinates(row):
    longitude, latitude = transformer.transform(row['e'], row['n'])
    return pd.Series([longitude, latitude], index=['longitude', 'latitude'])

# Apply the function to each row in the DataFrame
data[['longitude', 'latitude']] = data.apply(transform_coordinates, axis=1)


# Adding Kreise to the data

In [11]:
from shapely import wkt
from shapely.geometry import Point
from shapely.geometry.polygon import Polygon

In [12]:
kreis_df = pd.read_csv('C:/Users/simon/Desktop/Code/Zuri_wie_neu_dashboard/kreis_data/data/stzh_adm_stadtkreise_a.csv')
kreis_df.rename(columns={"bezeichnung": "kreis_name"}, inplace=True)

kreis_df.sort_values('name', inplace=True)
kreis_df.drop(['objid','entstehung'], axis=1, inplace=True)
kreis_df.reset_index(drop=True, inplace=True)
kreis_df['geometry'] = kreis_df['geometry'].apply(wkt.loads)


transformer = Transformer.from_crs("EPSG:2056", "EPSG:4326", always_xy=True)

# Function to transform coordinates
def transform_coordinates(row):
    longitude, latitude = transformer.transform(row['e'], row['n'])
    return pd.Series([longitude, latitude], index=['longitude', 'latitude'])


def transform_polygon(polygon):
    # Transform all coordinates in the polygon
    coords = list(polygon.exterior.coords)
    transformed_coords = [transformer.transform(x, y) for x, y in coords]
    return Polygon(transformed_coords)

#Translate coordinates to lat,lon
kreis_df['geometry'] = kreis_df['geometry'].apply(transform_polygon)  



# Convert df's lat/lon to Shapely Point objects
data['point'] = data.apply(lambda row: Point(row['longitude'], row['latitude']), axis=1)

# Function to classify each point based on polygons in kreis_df
def classify_point(point, polygons):
    for idx, polygon in enumerate(polygons):
        if polygon.contains(point):
            return idx
    return None


In [13]:
# Function to classify points based on polygons
def classify_point(row):
    for _, polygon_row in kreis_df.iterrows():
        if row['point'].within(polygon_row['geometry']):
            return polygon_row['name']
    return 'Unknown'

# Classify each point in the data
data['kreis_name'] = data.apply(classify_point, axis=1)

# Now 'data' DataFrame includes 'kreis_name' which indicates which kreis each point falls into.

In [14]:
# There are 5 points that are not in any kreis. We will remove them from the data.
data = data[data['kreis_name'] != 'Unknown']

In [15]:
df = data[['longitude', 'latitude', 'status', 'requested_datetime', 'updated_datetime', 'service_time', 'service_name', 'report_month', 'report_year', 'detail', 'media_url', 'service_notice','point','kreis_name']]
df

,longitude,latitude,status,requested_datetime,updated_datetime,service_time,service_name,report_month,report_year,detail,media_url,service_notice,point,kreis_name
0,8.484226,47.374038,fixed - council,2013-03-14 15:16:15,2013-04-12 07:59:30,688.720833,Strasse/Trottoir/Platz,3,2013,Auf dem Asphalt des Bürgersteigs hat es eine E...,No media,Diese Reparatur wird von uns in den kommenden ...,POINT (8.484226301305455 47.374037733747315),9
1,8.508193,47.395119,fixed - council,2013-03-14 15:17:57,2013-04-12 08:00:22,688.706944,Strasse/Trottoir/Platz,3,2013,Vermessungspunkt ist nicht mehr bündig mit dem...,No media,Diese Reparatur wird von uns in den kommenden ...,POINT (8.508192919637715 47.39511862093258),10
2,8.559589,47.408259,fixed - council,2013-03-15 09:14:16,2013-04-12 08:08:10,670.898333,Strasse/Trottoir/Platz,3,2013,Beim Trottoir sind einige Randsteine defekt un...,https://www.zueriwieneu.ch/photo/4.0.jpeg?bfbb...,Diese Reparatur wird von uns in den kommenden ...,POINT (8.559589406156757 47.40825905055263),12
3,8.521627,47.399131,fixed - council,2013-03-15 09:17:15,2013-04-12 08:09:05,670.863889,Strasse/Trottoir/Platz,3,2013,Auf dem Parkplatz beim Waidspital sind einige ...,https://www.zueriwieneu.ch/photo/5.0.jpeg?e309...,Diese Reparatur wird von uns in den kommenden ...,POINT (8.521627371871496 47.39913121917527),10
4,8.538891,47.375455,fixed - council,2013-03-15 10:36:53,2013-04-23 13:50:33,939.227778,Abfall/Sammelstelle,3,2013,Arbeitskiste ist rund herum verschmiert,https://www.zueriwieneu.ch/photo/6.0.jpeg?8e65...,Dieses Graffiti wird von uns in den kommenden ...,POINT (8.538890671691396 47.3754549474245),1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
54140,8.520284,47.403434,fixed - council,2024-09-04 16:29:01,2024-09-04 19:00:15,2.520556,Schädlinge,9,2024,Bei Waldspaziergang durch Tigermückenschwarm v...,https://www.zueriwieneu.ch/photo/60666.0.jpeg?...,Vielen Dank für Ihre Meldung. \n \n Wegen d...,POINT (8.520283960452161 47.40343443277984),10
54141,8.515094,47.379997,fixed - council,2024-09-04 16:43:28,2024-09-04 19:14:00,2.508889,Schädlinge,9,2024,Sehr geehrte Damen und Herren \n \n Sie wer...,https://www.zueriwieneu.ch/photo/60668.0.jpeg?...,Guten Abend \n \n Das glauben wir Ihnen ger...,POINT (8.515093730982437 47.379996803316224),4
54142,8.507043,47.352313,confirmed,2024-09-04 16:46:14,2024-09-04 17:22:04,0.597222,Grünflächen/Spielplätze,9,2024,"Flacher Wassertrog, ursprünglich zur Schuhrein...",No media,No response to report.,POINT (8.507043152473488 47.35231262125655),3
54143,8.524451,47.372560,fixed - council,2024-09-04 16:51:49,2024-09-04 16:53:07,0.021667,Abfall/Sammelstelle,9,2024,Diese KACK Velo versperrt seit Wochen den Velo...,https://www.zueriwieneu.ch/photo/60670.0.jpeg?...,Die Zweiradordnung wird Ihr Anliegen prüfen un...,POINT (8.524450819608292 47.372560228326954),4


In [17]:
kreis_list = np.sort(df['kreis_name'].unique())
kreis_list

array([1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12], dtype=object)

In [18]:
import matplotlib
matplotlib.rcParams['figure.figsize'] = (16, 10)

# Predict reports

### Split data into train, val, test based on time.

In [ ]:
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Input
from tensorflow.keras.callbacks import EarlyStopping
# Prepare the data
df1 = df[['requested_datetime', 'service_name']].copy()
# Extract time-based features
df1.loc[:,'day'] = df1['requested_datetime'].dt.day
df1.loc[:,'month'] = df1['requested_datetime'].dt.month
df1.loc[:,'year'] = df1['requested_datetime'].dt.year
df_daily_counts = df1.groupby(['year', 'month', 'day']).size().reset_index(name='count')
df_daily_counts['date'] = pd.to_datetime(df_daily_counts[['year', 'month', 'day']])
df_daily_counts = df_daily_counts[['date', 'count']]
# Generate a complete date range
date_range = pd.date_range(start=df_daily_counts['date'].min(), end=df_daily_counts['date'].max())

# Reindex the DataFrame to include all dates
df_daily_counts = df_daily_counts.set_index('date').reindex(date_range).fillna(0).rename_axis('date').reset_index()
df_daily_counts


train_split_end = '2022-01-01'
val_split_end   = '2023-01-01'


# Train, validation, and test splits
train_data = df_daily_counts[df_daily_counts['date'] < train_split_end]
val_data  = df_daily_counts[(df_daily_counts['date'] >= train_split_end) & (df_daily_counts['date'] < val_split_end)]
test_data = df_daily_counts[df_daily_counts['date'] >= val_split_end]

# Drop 'date' as it is not a feature for prediction
train_data = train_data.drop(columns=['date'])
val_data   = val_data.drop(columns=['date'])
test_data  = test_data.drop(columns=['date'])

# Scaling the data
scaler = MinMaxScaler()
train_scaled = scaler.fit_transform(train_data)
val_scaled   = scaler.transform(val_data)
test_scaled  = scaler.transform(test_data)

# Create sequences for LSTM
def create_sequences(data, sequence_length):
    X, y = [], []
    for i in range(len(data) - sequence_length):
        X.append(data[i:i+sequence_length])
        y.append(data[i+sequence_length])
    return np.array(X), np.array(y)

# Define sequence length
sequence_length = 32

# Create sequences
X_train, y_train = create_sequences(train_scaled, sequence_length)
X_val,   y_val   = create_sequences(val_scaled, sequence_length)
X_test,  y_test  = create_sequences(test_scaled, sequence_length)

# Define the LSTM model
model = Sequential([
    Input(shape=(sequence_length, 1)),
    LSTM(50, return_sequences=True),
    LSTM(50),
    Dense(1)
])

model.compile(optimizer='adam', loss='mse')

# Train the model
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
history = model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=100, callbacks=[early_stopping])

# Make predictions
predictions = model.predict(X_test)

# Inverse transform the predictions
predictions = scaler.inverse_transform(predictions)

# Create the result DataFrame
predicted_dates = df_daily_counts['date'].iloc[-len(predictions):].reset_index(drop=True)
predicted_counts = pd.DataFrame(predictions, columns=['predicted_number_of_reports'])

result_df = pd.concat([predicted_dates, predicted_counts], axis=1)

In [ ]:
plt.figure(figsize=(14, 7))

# Plot actual counts
plt.plot(df_daily_counts['date'], df_daily_counts['count'], label='Actual Counts', color='blue')

# Plot predicted counts
mask = (result_df['date'] >= '2023-01-01')
plt.plot(result_df[mask]['date'], result_df[mask]['predicted_number_of_reports'], label='Predicted Counts', color='red')

# Add title and labels
plt.title('Actual vs. Predicted Number of Reports (Full Range)')
plt.xlabel('Date')
plt.ylabel('Number of Reports')
plt.legend()
plt.savefig('Forecast_2013_2024.png', bbox_inches='tight')

# Show the plot
plt.show()

# Zoomed-in plot for 2022-2024
plt.figure(figsize=(14, 7))

# Plot actual counts
plt.plot(df_daily_counts['date'], df_daily_counts['count'], label='Actual Counts', color='blue')

# Plot predicted counts
plt.plot(result_df['date'], result_df['predicted_number_of_reports'], label='Predicted Counts', color='red')

# Add title and labels
plt.title('Actual vs. Predicted Number of Reports (2022-2024)')
plt.xlabel('Date')
plt.ylabel('Number of Reports')
plt.legend()

# Set x-axis limits
plt.xlim(pd.Timestamp('2023-01-01'), pd.Timestamp('2024-09-05'))
plt.savefig('Forecast_2023_2024.png', bbox_inches='tight')
# Show the plot
plt.show()

## Predict the future

Autoregressive analysis. The LSTM takes a sequence of reports and predicts the next report. To predict the future, we can use our predictions as input. E.g. we use the past week, today (T) minus 7 to T-1, to predict today. Then we use T-6 to T-1 + our prediction of T to predict T+1.

In [ ]:
# Number of days to predict beyond the test set
n_future_days = int(6*30)  # For example, predict the next 30 days

# Create an input sequence for future prediction starting from the last test data
last_sequence = test_scaled[-sequence_length:]  # Last 'sequence_length' days from the test data

# Initialize a list to hold future predictions
future_predictions = []

# Autoregressive prediction for future days
for _ in range(n_future_days):
    # Predict the next day based on the current input sequence
    next_prediction = model.predict(last_sequence[np.newaxis, :, :])
    
    # Append the predicted value to the future predictions
    future_predictions.append(next_prediction[0, 0])
    
    # Update the input sequence for the next iteration
    # Remove the first element and append the predicted value to the end
    last_sequence = np.append(last_sequence[1:], next_prediction, axis=0)

# Inverse transform the future predictions back to the original scale
future_predictions = scaler.inverse_transform(np.array(future_predictions).reshape(-1, 1))

# Generate future dates starting from the last date in your test set
last_test_date = df_daily_counts['date'].max()
future_dates = pd.date_range(start=last_test_date + pd.Timedelta(days=1), periods=n_future_days)

# Create a DataFrame for the future predictions
future_predictions_df = pd.DataFrame({
    'date': future_dates,
    'predicted_number_of_reports': future_predictions.flatten()
})

# Concatenate the future predictions with the existing result DataFrame
extended_result_df = pd.concat([result_df, future_predictions_df], ignore_index=True)

# Display the extended predictions


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# Set seaborn style for better visuals
sns.set(style="whitegrid")

# Plot the results
plt.figure(figsize=(12, 6))

# Plot the actual test data
plt.plot(df_daily_counts['date'].iloc[-len(test_data):], 
         scaler.inverse_transform(test_scaled), 
         label='Actual Test Data', color='blue')

# Plot the predictions on the test set
plt.plot(result_df['date'], 
         result_df['predicted_number_of_reports'], 
         label='Test Set Predictions', color='green', linestyle='dashed')

# Plot the future predictions
plt.plot(future_predictions_df['date'], 
         future_predictions_df['predicted_number_of_reports'], 
         label='Future Predictions', color='red', linestyle='dashed')

# Add labels and title
plt.title('Test Set Predictions and Future Predictions')
plt.xlabel('Date')
plt.ylabel('Number of Reports')
plt.xticks(rotation=45)

# Add legend
plt.legend()

# Display the plot
plt.tight_layout()
plt.show()
